In [1]:
import warnings
warnings.filterwarnings('ignore')
from textgenrnn import textgenrnn
import syllapy

Download poetry collection to insert some culture into this model

In [2]:
download_fn = 'poetry.ndjson.gz'
! mkdir -p data
! rm -f data/$download_fn
! curl -o data/$download_fn http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52.2M  100 52.2M    0     0  40.3M      0  0:00:01  0:00:01 --:--:-- 40.3M


In [3]:
poetry_fn = download_fn.replace('.gz', '')
! rm -f data/$poetry_fn
! gzip -d data/$download_fn

In [4]:
long_fn = 'poetry.txt'
short_fn = 'poetry_short.txt'
with open(f'data/{poetry_fn}', 'r') as infile, open(f'data/{long_fn}', 'w') as outfile:
    for line in infile:
        nice_text = line.replace('{"s": "', '').split('", "gid"')[0].replace('\\', '')
        outfile.write(nice_text + '\n')
! shuf data/$long_fn -n 10000 > data/$short_fn

Train RNN on subset of poetry

In [5]:
textgen = textgenrnn()
textgen.train_from_file(f'data/{short_fn}', num_epochs=1)

10,000 texts collected.
Training on 396,486 character sequences.
3097/3097 [==============================] - 437s 140ms/step - loss: 1.8346
####################
Temperature: 0.2
####################
And the first to the word strength the bear

And the both of the word of the crowd

The strange was bear thou the bear of the both,

####################
Temperature: 0.5
####################
And that the boys of him and sweet flames

That the glad the whole whose field is the very way,

The man shall by my both thou and not,

####################
Temperature: 1.0
####################
they cruin plat, though the way?

Are those whodrown heart, eat if thy day,

Son to myth Clock hundness, dermid down,



Generate random lines of poetry and keep those with a reasonable number of syllables

In [6]:
a = textgen.generate(100, temperature=0.5, return_as_list=True)

100%|██████████| 100/100 [03:42<00:00,  2.23s/it]


In [7]:
l_filtered = []
for l in a:
    length = syllapy.count(l)
    if 8 <= length <= 12:
        l_filtered.append(l)

In [8]:
class Poem:
    def __init__(self, lines):
        '''https://en.wikipedia.org/wiki/Sestina'''
        assert isinstance(lines, list)
        assert len(lines) >= 39
        
        self.end_words = {i:'' for i in range(6)}
        
        # I subtract 1 from each word index to be consistent with python's
        # zero-based indexing, but I start with the 1-based to be consistent with 
        # wikipedia documentation
        self.rotation = [6, 1, 5, 2, 4, 3, \
                         3, 6, 4, 1, 2, 5, \
                         5, 3, 2, 6, 1, 4, \
                         4, 5, 1, 3, 6, 2, \
                         2, 4, 6, 5, 3, 1]
        self.rotation = [x - 1 for x in self.rotation]
        self.envoi = [(6, 2), (1, 4), (5, 3)]
        self.envoi = [(t[0]-1, t[1]-1) for t in self.envoi]
        
        self.sestina = []
        self.get_end_words(lines[:6])
        self.replace_last_words(lines[6:36])
        self.compose_envoi(lines[36:39])
    
    def get_end_words(self, first_6_lines):
        assert len(first_6_lines) == 6
        for i, line in enumerate(first_6_lines):
            self.end_words[i] = line.split()[-1]
        self.sestina.extend(first_6_lines)
    
    def replace_last_words(self, middle_lines):
        assert len(middle_lines) == 30
        for i, line in enumerate(middle_lines):
            word_i = self.rotation[i]
            new_line = line.split()[:-1] + [self.end_words[word_i]]
            self.sestina.append(' '.join(new_line))
    
    def compose_envoi(self, last_lines):
        assert len(last_lines) == 3
        for i, line in enumerate(last_lines):
            pts = line.split()[:-1]
            half = len(pts) // 2
            etpl = self.envoi[i]
            new_line = pts[:(half-1)] + [self.end_words[etpl[0]]] + \
                       pts[half:-1] + [self.end_words[etpl[1]]]
            self.sestina.append(' '.join(new_line))
    
    def recite(self):
        for i, line in enumerate(self.sestina):
            print(line)
            if (i+1)%6 == 0 and i > 0:
                print()


In [9]:
p = Poem(l_filtered)
p.recite()

And silver who told the mouthes;
And the clothed black strive his book
What son who were not all the gain word sea,
And folk of just the live the constran thought,
Same, once of the gold of fair,
And the lets he had been believed.

And some who beat the son of the believed.
The cold should go and hand of the mouthes;
Though the which fille the breather of the fair,
To the fild seas, the crime of the book
And made it the second through the old thought,
And do the bear silves; and sea,

And the water disparking bear the sea,
And a flight swind of the believed.
And words on the shame the rcought of the thought,
And all the mountains of the mouthes;
You might the rock of the arming book
And the glory thought brook to the fair,

Is the deflisting time of all fair,
The band showed he old brooks the sea,
And the boneves her door of the book
And who worth the rest heart, the crime of the believed.
One look the sumbers of the mouthes;
That the sings were blooms the low strokes thought,

The war